# Metabolic model quality check and improvement workflow

1.Download the initila model constructed from ModelSEED. The format of initial model is SBML file.


2.generate an excel file for the model for easy checking. 

In [1]:
#2 excel format model generation. for checking and revising model, such as reactions and their lower or upper bound information.
from metnet import sbml_excel
inputforder='/media/jupyter/data/'
sbml_excel(inputforder+'Seed1111.5.65232.xml','Seed1111.5.65232.xlsx')# sbml to excel
#sbmltoexcel.model_excel(model_1,'model_1.xlsx') # cobrapy model to excel. If we want to check the model we can change it to excel formate at any time

3 initial check and model revision

In [175]:
#3.1 import some modules and read the sbml model
from __future__ import print_function
import cobra
import re
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import pfba
inputforder='/media/jupyter/data/'
model = cobra.io.read_sbml_model(inputforder+'Seed1111.5.65232.xml') # read sbml format model

In [176]:
#3.2 Calculate optimal growth rate and check its input and output
model.optimize()
model.summary() #check input/outputs for the optimal solution
#model.medium
# many substrates can be consumed, leading to unreasonable high growth rates, need to modify the boundaries of exchange reactions

IN FLUXES             OUT FLUXES            OBJECTIVES
--------------------  --------------------  -------------
cpd00162_e0    1e+03  cpd00001_e0    1e+03  biomass0  149
cpd00794_e0    1e+03  cpd00011_e0    1e+03
cpd00130_e0  692      cpd00067_e0    1e+03
cpd00129_e0  403      cpd00221_e0    1e+03
cpd00307_e0  358      cpd00142_e0  969
cpd00007_e0  184      cpd00013_e0  900
cpd00009_e0  121      cpd00588_e0  479
cpd00075_e0  110      cpd00036_e0  352
cpd00039_e0   46.3    cpd00092_e0  327
cpd00226_e0   45.1    cpd00100_e0  311
cpd00051_e0   36.8    cpd00023_e0  282
cpd00107_e0   36.7    cpd00156_e0  189
cpd00322_e0   36.1    cpd00276_e0   94.6
cpd00082_e0   34.3
cpd00105_e0   30.9
cpd11581_e0   29.9
cpd00066_e0   23
cpd11591_e0   19.5
cpd15604_e0   19.3
cpd15606_e0   18
cpd01017_e0   12.7
cpd01080_e0   12.7
cpd00119_e0   11.8
cpd00065_e0    7.04
cpd03847_e0    3.73
cpd10515_e0    1.85
cpd00393_e0    1.39
cpd11606_e0    0.924
cpd00030_e0    0.462
cpd00034_e0    0.462
cpd00048_e0    0.4

In [177]:
#3.3 Change boundary of exchange reactions
'''
1. mark exchange reactions in excel file, mark exchange reactions for carbon,oxygen,nitrogen ions etc
2. normally only allow one carbon source (default was glucose), uptake rate was set at 10mmol/g/h.
3. oxygen exchange rate can be set to 0 for anaerobic or unlimited for aerobic
4. CO2 exchange rate can be negative for CO2 fixation or 0 so that CO2 can only be released
5. nitrogen (NH3) sulfur (H2S2O3,sulfate) and phosphate (Pi) sources can also be set, normally not limited
6. exchange reactions for ions(Pb,Cd2,Cu2,Fe2,Fe3,Mg,K,Zn2,Na,Co2,Cl,Hg2,Mn2,Ca2) and H2O were unconstrained.
7. boundaries of all other exchange reactions were set to 0-1000 so that the metabolites can only be produced but not consumed. 
'''
c_exchange=['EX_cpd00027_e0'] #exchange reaction for carbon sources, use list to allow multiple sources
for ex in c_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-10, 1000) # maximal uptake rate 10mmol/g/h
o_exchange=['EX_cpd00007_e0'] #exchange reaction for oxygen for anaerobic or aerobic conditions
for ex in o_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
co2_exchange=['EX_cpd00011_e0'] #exchange reaction for CO2, 
for ex in co2_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(0, 1000) 
n_exchange=['EX_cpd00013_e0'] #exchange reaction for N sources
for ex in n_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
s_exchange=['EX_cpd00048_e0','EX_cpd00268_e0'] #exchange reaction for sulafte and H2S2O3
for ex in s_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
p_exchange=['EX_cpd00009_e0','EX_cpd00012_e0'] #exchange reaction for Pi and PPi
for ex in p_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
ion_exchange=['EX_cpd04097_e0','EX_cpd00001_e0','EX_cpd00058_e0','EX_cpd10515_e0','EX_cpd10516_e0','EX_cpd00254_e0','EX_cpd00205_e0',
            'EX_cpd00034_e0','EX_cpd00971_e0','EX_cpd00149_e0','EX_cpd00099_e0','EX_cpd00531_e0','EX_cpd00030_e0','EX_cpd00063_e0']
for ex in ion_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
preset_exchange=c_exchange+o_exchange+co2_exchange+n_exchange+s_exchange+p_exchange+ion_exchange
for rea in model.boundary: 
    if rea.id not in preset_exchange: #all other exchange reastions can only be outputs
        rea.bounds=(0,1000)
model.optimize()
model.summary() 
cobra.io.write_sbml_model(model, "model01.xml")
#growth likely to be zero, gaps in the network, but first check ATP production

IN FLUXES        OUT FLUXES       OBJECTIVES
---------------  ---------------  ------------------
cpd00007_e0  60  cpd00011_e0  60  biomass0  1.39e-28
cpd00027_e0  10  cpd00001_e0  60


4 Check and revise the ATP production pathway 

In [178]:
#4.1 
'''
we often get very high ATP production rates caused by wrong ATP generation cycle, need to check the pathways one by one to correct
the mistakes using the ATP consumption reaction (maintenance) rxn00062 as objective reaction
find the maintenance reaction by searching in excel, should change the lower bound to maintenace coefficient such as 4.9 
'''
#rin=model.reactions.get_by_id("EX_cpd00027_e0") # check the glucose input
#rin.bounds=(0, 1000) # we can turn off the carbon input for checking wrong ATP generation pathways
if "rxn00062_c0" not in model.reactions: # rxn00062_c0 is the maintenance reaction of the model, if rxn00062_c0 not in model, add this reaction in model
    #reaction = Reaction('ATPM')
    reaction = Reaction('rxn00062_c0')
    model.add_reactions([reaction])
    reaction.name = 'ATP maintenance reaction'
    reaction.build_reaction_from_string('cpd00001_c0 + cpd00002_c0 => cpd00008_c0 + cpd00009_c0 + cpd00067_c0')
    model.add_reactions([reaction])
#else:
    #model.reactions.get_by_id("rxn00062_c0").id='ATPM'
atpm=model.reactions.get_by_id("rxn00062_c0")
atpm.bounds=(4.9, 1000) #maintenace rate as lower bound
#atpm.bounds=(-1000, 1000) #修改以使其可最小化净生成ATP，从而可计算净消耗ATP的无效循环
model.objective=atpm # change objective function
model.optimize()
model.summary() 

IN FLUXES        OUT FLUXES       OBJECTIVES
---------------  ---------------  ------------------
cpd00007_e0  60  cpd00011_e0  60  rxn00062_c0  1e+03
cpd00027_e0  10  cpd00001_e0  60


In [136]:
#4.2 output the pathway to a text file to find the mistakes
from metnet import pathway
outputfile_name ='atp.txt' # provide output file name
fluxes = pfba(model).fluxes #use pfba to get a simple path，you should provide your model name
pathway(model,fluxes,outputfile_name)

By checking the result pathway file, we can find ATP without consumption of any substrate but from a cycle pathway. Reaction flux for rxn00058_c0 ((4.0)H_c0 + (4.0)Cytochrome_c2_c0 + O2_c0 -->(2.0)H2O_c0 + (4.0)Cytochrome_c3_c0) is -500, however this reaction	should be irreversible as O2_c0 should only be consumed but not produced in the respiratory chain. This should be corrected by change the lower bound of this reaction to zeor to make it irreversible. 
Actually most errors in ATP generation were caused by mistakes in reaction reversibility, we need to repeat the process untill all erros are corrected and a reasonable amount of ATP is produced from the carbon source.We can use of a set of rules to quickly determine which reaction in a wrong pathway should be set to irreversible.
1.	Oxygen consumption reactions,
2.	Most of the carbon dioxide production reaction (CO2 is only used as substrate when a high energy substrate such as PEP (phophoenolpyruvate) and ATP is consumed at the same time),
3.	Most of the NH3 production reaction (NH3 is only used in two NH3 assimilation reactions and carbamoyl phosphate production),
4.	Most of the phosphate production reaction (the reaction is regarded as reversible only when phosphate reacts with another high energy substrate such as AcCoA)
5.	Reactions in which S-Adenosyl-L-methionine is converted to S-Adenosyl-L-homocystine for providing a methyl group,
6.	Reactions in which tetrahydrofolate (THF) is produced for transferring one carbon unit, 
7.	Most of the ATP (or other high energy metabolites) consumption reactions, except for reactions with another high energy metabolite such as GTP, CAP, acetyl phosphate, Acyl-CoA,
8.	UDP-sugar consumption reactions for transferring sugar units,  
9.	CDP-diacylglycerol consumption reactions for phosphatidyl group transfer,
10.	reactions like: 3'-Phosphoadenylylsulfate (PAPS) + A = adenosine 3',5'-bisphosphate (PAP) + B

To avoid the time consuming correction process, we compiled a list of reactions from ModelSeed that should be corrected to irreversible to avoid wrong ATP generation pathways. The model can be updated from this list. 
请生成一个修正反应列表，文本文件含三列数据 反应ID，下界 上界，由此文件更新sbml文件的model02.


In [35]:
#4.3 Change the boundary of the model according to this list , then wo got the model 2.0 version
import csv
s = csv.reader(open(inputforder+'PST_for_ATP_list.txt'), delimiter="\t")
realist = []
realist.extend(s)
for j in realist:
    model.reactions.get_by_id(j[0]).bounds=(float(j[1]),float(j[2]))
cobra.io.write_sbml_model(model, "model02.xml")
# After running this step, go to run 4.2 to check the pathway of ATP synthesis.

In [179]:
#4.3 Change the boundary of the model according to revised ModelSEED database , then wo got the model 2.0 version
from metnet import compare
#model=cobra.io.read_sbml_model("model02.xml")
model_seed = cobra.io.read_sbml_model(inputforder+'seed_1.xml') # read sbml format model
outputfile_name ='Different.txt' # provide output file name
compare(model,model_seed,outputfile_name)
cobra.io.write_sbml_model(model, "model02.xml")
# After running this step, go to run 4.2 to check the pathway of ATP synthesis

Bounds not same	rxn01199_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn08851_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn01675_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn12644_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn08800_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn08823_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn03419_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn10125_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn05734_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn12637_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000

Bounds not same	rxn08802_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn08848_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn00650_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn12646_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn10236_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn05465_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn10222_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn00392_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn02774_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn08708_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000

Bounds not same	rxn02000_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn08843_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn12635_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn00213_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn08528_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn00679_c0	my model is: (-1000.0, 1000.0)	Database is: (-1000.0, 0.0)	Already changed
Bounds not same	rxn08797_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn01102_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn09198_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Bounds not same	rxn01671_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 100

Reaction is not in SEED database	EX_cpd00063_e0
Reaction is not in SEED database	EX_cpd00793_e0
Reaction is not in SEED database	EX_cpd00393_e0
Reaction is not in SEED database	EX_cpd00030_e0


In [162]:
#4.3 Change the boundary of the model according to revised ModelSEED database , then wo got the model 2.0 version
model_seed = cobra.io.read_sbml_model(inputforder+'seed_1.xml') # read sbml format model
outputfile_name ='Different.txt' # provide output file name
flux = open(outputfile_name, 'w')
for i in model.reactions: #compare our model with curated ModelSEED 
    equal=0
    for j in model_seed.reactions:
        if i.id ==j.id:
            equal=1
            model_reactants=[]
            modelseed_reactants=[]
            model_products=[]
            modelseed_products=[]                                                                               
            current=['cpd00067_c0']#反应方程是否一致的检查不包括流通代谢物的检查，引起最多不一致的是H_c，后边再继续加
            for m_rea in range(len(i.reactants)):
                if not str(i.reactants[m_rea]) in current: 
                    model_reactants.append(str(i.reactants[m_rea]))
            for m_pro in range(len(i.products)):
                if not str(i.products[m_pro]) in current:
                    model_products.append(str(i.products[m_pro]))
            for ms_rea in range(len(j.reactants)):
                if not str(j.reactants[ms_rea]) in current:
                    modelseed_reactants.append(str(j.reactants[ms_rea]))
            for ms_pro in range(len(j.products)):
                if not str(j.products[ms_pro]) in current:
                    modelseed_products.append(str(j.products[ms_pro]))
            if set(model_reactants)==set(modelseed_reactants) and set(model_products)==set(modelseed_products):# check if reactions is same
                if not i.bounds == j.bounds: # check if bounds is same
                    print('Bounds not same'+'\t'+i.id+'\t'+'my model is: '+str(i.bounds)+'\t'+'Database is: '+str(j.bounds)+'\t'+'Already changed')
                    flux.write('Bounds not same'+'\t'+i.id+'\t'+'my model is: '+str(i.bounds)+'\t'+'Database is: '+str(j.bounds)+'\t'+'Already changed'+'\n')
                    i.bounds = j.bounds #直接把库中可逆性付给模型
            if set(model_reactants)==set(modelseed_products) and set(model_products)==set(modelseed_reactants): # check if reactions is reverse
                if not i.lower_bound==-j.upper_bound or not i.upper_bound==-j.lower_bound:
                    print('Reaction inversion'+'\t'+i.id+'\t'+'my model is: '+str(i.bounds)+'\t'+'Database is: '+str(j.bounds)+'\t'+'Already changed')
                    flux.write('Reaction inversion'+'\t'+i.id+'\t'+'my model is: '+str(i.bounds)+'\t'+'Database is: '+str(j.bounds)+'\t'+'Already changed'+'\n')
                    i.lower_bound=-j.upper_bound
                    i.upper_bound==-j.lower_bound
            if set(model_reactants)==set(modelseed_reactants) and not set(model_products)==set(modelseed_products):
                print('Products is not same'+'\t'+i.id+'\t'+'my model products is: '+ '\t'+str(model_products) +'SEED products is: '+ str(modelseed_products))
                flux.write('Products is not same'+'\t'+i.id+'\t'+'my model products is: '+ '\t'+str(model_products) +'SEED products is: '+ str(modelseed_products)+'\n')
            if not set(model_reactants)==set(modelseed_reactants) and set(model_products)==set(modelseed_products):
                print('Reactants is not same'+'\t'+i.id+'\t'+'my model reactants is: '+ '\t'+str(model_reactants) +'SEED reactants is: '+ str(modelseed_reactants))
                flux.write('Reactants is not same'+'\t'+i.id+'\t'+'my model reactants is: '+ '\t'+str(model_reactants) +'SEED reactants is: '+ str(modelseed_reactants)+'\n')
    if equal==0:
        print('Reaction is not in SEED database'+'\t'+i.id)
        flux.write('Reaction is not in SEED database'+'\t'+i.id+'\n')
flux.close()
cobra.io.write_sbml_model(model, "model02.xml")
# After running this step, go to run 4.2 to check the pathway of ATP synthesis

Reaction inversion	rxn00704_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Reaction inversion	rxn01265_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Reaction inversion	rxn02003_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Reaction is not in SEED database	rxn12664_e0
Reaction is not in SEED database	rxn05514_e0
Reaction inversion	rxn01541_c0	my model is: (-1000.0, 1000.0)	Database is: (0.0, 1000.0)	Already changed
Reaction is not in SEED database	rxn05722_e0
Reaction is not in SEED database	rxn05255_e0
Reaction is not in SEED database	rxn05618_e0
Reaction is not in SEED database	biomass0
Reaction is not in SEED database	EX_cpd11590_e0
Reaction is not in SEED database	EX_cpd16062_e0
Reaction is not in SEED database	EX_cpd00067_e0
Reaction is not in SEED database	EX_cpd00156_e0
Reaction is not in SEED database	EX_cpd11583_e0
Reaction is not in SEED database	EX_cpd01242_e0
Reaction is not in SEED d

In [180]:
#5.1 Check the optimal growth rate of this model.
model.objective=model.reactions.get_by_id('biomass0') # change objective function
model.optimize()
model.summary() 

IN FLUXES        OUT FLUXES       OBJECTIVES
---------------  ---------------  -------------------
cpd00007_e0  60  cpd00001_e0  60  biomass0  -5.29e-29
cpd00027_e0  10  cpd00011_e0  60


In [8]:
#5.2 If the growth rate was 0, calculate every biomass precursor to check which matabolite can not be generated
# mark exchange reactions in excel file, mark exchange reactions for carbon,oxygen,nitrogen ions etcexclude_mets contain all metabolites that can be directly exculded by this model. 
#output the precursor that cannot be synthesized
from metnet import calculate, mkdir
#import makefolder
#import biomass_cal
outfolder = './biomass_precursors/' # (*Should provide) Folder's for store all calculated results of biomass precursors
mkdir (outfolder) # Create Folder
summary = outfolder + 'summary.txt'
#exclude_mets=['cpd00254_c0','cpd00205_c0','cpd17042_c0','cpd00034_c0','cpd00063_c0','cpd17041_c0','cpd00099_c0','cpd00149_c0','cpd17043_c0','cpd00030_c0','cpd00048_c0','cpd10515_c0','cpd10516_c0','cpd00058_c0','cpd00001_c0']
precursors=[]
for i in model.reactions.get_by_id('biomass0').reactants: # The biomass ID of initial model usually contains 'bio'
    #if str(i) not in exclude_mets:
        precursors.append(i)
calculate(precursors,model,outfolder,summary) # Calculate all precursors' optimal production rtae

./biomass_precursors/ 目录已存在
TPP_c0
Putrescine_c0
2_Demethylmenaquinone_8_c0
ACP_c0
Diisoheptadecanoylphosphatidylglycerol_c0
Peptidoglycan_polymer_n_subunits_c0
Anteisoheptadecanoylcardiolipin_B_subtilis_c0
5_Methyltetrahydrofolate_c0
Calomide_c0
L_Lysine_c0
Dianteisoheptadecanoylphosphatidylglycerol_c0
L_Phenylalanine_c0
10_Formyltetrahydrofolate_c0
Menaquinone_8_c0
Isoheptadecanoylcardiolipin_B_subtilis_c0
L_Arginine_c0
Tetrahydrofolate_c0
phosphatidylethanolamine_dioctadecanoyl_c0
L_Tyrosine_c0
Phosphatidylglycerol_dioctadecanoyl_c0
Siroheme_c0
Dianteisoheptadecanoylphosphatidylethanolamine_c0
Ubiquinone_8_c0
Spermidine_c0
Diisoheptadecanoylphosphatidylethanolamine_c0
Stearoylcardiolipin_B_subtilis_c0


In [9]:
#5.3 Calculate the biomass precursors of manually corrected model to check synthetic pathways of these nonsynthetic metabolite
from metnet import calculate, mkdir
import csv
import cobra
from cobra import Model, Reaction, Metabolite
inputfolder = '/media/jupyter/data/' # (*Should provide) Folder's for storing all calculated results of biomass precursors
outfolder = './PST_mets/'
mkdir (outfolder) # Create Folder
summary = outfolder + 'summary_pstmodel.txt'
modelpst = cobra.io.read_sbml_model(inputfolder+'PST_model.xml') # read sbml format model
reaction = Reaction('bio')
modelpst.add_reactions([reaction])
reaction.build_reaction_from_string('35.5403092430435 H2O_c0 + 40.1101757365074 ATP_c0 + 0.00309646685192537 NAD_c0 + 0.00309646685192537 NADP_c0 + 0.00309646685192537 CoA_c0 + 0.00309646685192537 FAD_c0 + 0.00309646685192537 Pyridoxal_phosphate_c0 + 0.00309646685192537 S_Adenosyl_L_methionine_c0 + 0.219088153012743 L_Glutamate_c0 + 0.00309646685192537 Heme_c0 + 0.00309646685192537 Mn2_c0 + 0.509869786991038 Glycine_c0 + 0.00309646685192537 Zn2_c0 + 0.427934380173264 L_Alanine_c0 + 0.135406821203723 GTP_c0 + 0.285438020490179 L_Lysine_c0 + 0.200830806928348 L_Aspartate_c0 + 0.00309646685192537 GSH_c0 + 0.00309646685192537 Sulfate_c0 + 0.246696822701341 L_Arginine_c0 + 0.219088153012743 L_Glutamine_c0 + 0.179456352975885 L_Serine_c0 + 0.00309646685192537 TPP_c0 + 0.00309646685192537 Cu2_c0 + 0.127801422590767 L_Methionine_c0 + 0.0908319049068452 UTP_c0 + 0.00309646685192537 Ca2_c0 + 0.0472019191450218 L_Tryptophan_c0 + 0.154519490031345 L_Phenylalanine_c0 + 0.120676604606612 L_Tyrosine_c0 + 0.0761464922056484 L_Cysteine_c0 + 0.00309646685192537 Tetrahydrofolate_c0 + 0.00309646685192537 Cl__c0 + 0.375388847540127 L_Leucine_c0 + 0.0320579110651499 dATP_c0 + 0.00309646685192537 Putrescine_c0 + 0.0792636000737159 L_Histidine_c0 + 0.184354665339991 L_Proline_c0 + 0.200830806928348 L_Asparagine_c0 + 0.00309646685192537 Co2_c0 + 0.352233189091625 L_Valine_c0 + 0.211072732780569 L_Threonine_c0 + 0.00309646685192537 Calomide_c0 + 0.00309646685192537 10_Formyltetrahydrofolate_c0 + 0.00309646685192537 K_c0 + 0.00309646685192537 Riboflavin_c0 + 0.00309646685192537 Mg_c0 + 0.00309646685192537 Spermidine_c0 + 0.241798510337235 L_Isoleucine_c0 + 0.00309646685192537 5_Methyltetrahydrofolate_c0 + 0.0320579110651499 TTP_c0 + 0.00309646685192537 Siroheme_c0 + 0.0250105977108944 Bactoprenyl_diphosphate_c0 + 0.00309646685192537 Fe2_c0 + 0.00309646685192537 fe3_c0 + 0.00309646685192537 ACP_c0 + 0.00309646685192537 2_Demethylmenaquinone_8_c0 + 0.0250105977108944 core_oligosaccharide_lipid_A_c0 + 0.00309646685192537 Menaquinone_8_c0 + 0.0106480421341882 phosphatidylethanolamine_dioctadecanoyl_c0 + 0.0106480421341882 Phosphatidylglycerol_dioctadecanoyl_c0 + 0.00309646685192537 Ubiquinone_8_c0 + 0.0250105977108944 Peptidoglycan_polymer_n_subunits_c0 + 0.0106480421341882 Diisoheptadecanoylphosphatidylethanolamine_c0 + 0.0106480421341882 Dianteisoheptadecanoylphosphatidylethanolamine_c0 + 0.0106480421341882 Diisoheptadecanoylphosphatidylglycerol_c0 + 0.0106480421341882 Dianteisoheptadecanoylphosphatidylglycerol_c0 + 0.0106480421341882 Stearoylcardiolipin_B_subtilis_c0 + 0.0106480421341882 Isoheptadecanoylcardiolipin_B_subtilis_c0 + 0.0106480421341882 Anteisoheptadecanoylcardiolipin_B_subtilis_c0 + Protein_biosynthesis_c0 + DNA_replication_c0 + RNA_transcription_c0 --> 40.0 ADP_c0 + 39.9969035331481 Phosphate_c0 + 0.484633900402731 PPi_c0 + 40.0 H_c0 + 0.00309646685192537 Dimethylbenzimidazole_c0 + 0.00309646685192537 Cobinamide_c0 + Biomass_c0 + 0.00309646685192537 apo_ACP_c0 + 0.0250105977108944 Peptidoglycan_polymer_n_1_subunits_c0')
precursors=[]
for i in modelpst.reactions.get_by_id('bio').reactants:     
    precursors.append(i)
#biomass_cal.calculate(precursors,model,outfolder,summary) # Calculate all precursors' optimal production rtae
writefile = open(summary, 'w')
with modelpst:
    for i in precursors:
        demand = modelpst.add_boundary(i,type='demand')
        modelpst.objective = demand
        outputfile_name = outfolder + str(i) + '.txt'
        fluxes = cobra.flux_analysis.pfba(modelpst).fluxes
        a='DM_'+ str(i)
        obj=str(fluxes[a])
        writefile.write(str(i)+"\t"+obj+"\n")                  
        flux = open(outputfile_name, 'w')
        for r,v in fluxes.iteritems():
            if abs(v)>1e-6:
            #print (r,v)
                for line in modelpst.reactions:
                    if line.id==r:
                        m=str(modelpst.reactions.get_by_id(r))
                        n=m.split(':')
                        flux.write(r +'\t' + str(round(v,4)) + '\t' + n[1] +'\n')
        flux.close()
writefile.close()

./PST_mets/ 创建成功
unknown metabolite 'Peptidoglycan_polymer_n_subunits_c0' created
unknown metabolite 'Protein_biosynthesis_c0' created
unknown metabolite 'Peptidoglycan_polymer_n_1_subunits_c0' created


In [181]:
#5.4 Gap check for seed model, according to the calculated results of curated model finding the gap.
inputfolder = './PST_mets/' # Folder's storing all calculated results of biomass precursors
check_pre=inputfolder+'L_Lysine_c0.txt' # the precursors we want to check 
s=csv.reader(open(check_pre),delimiter="\t")
realist=[]
realist.extend(s)
mets={} #字典存放代谢物ID和全称对应列表
for m in model.metabolites:
    mets[m.name]=m
for i in realist:
    rea={} #存放模型中存在的反应物
    rea1={}#存放模型中存在的产物
    rea0={} #存放模型中不存在的反应物
    rea11={}#存放模型中不存在的产物
    have0=0 #反应物有不能生成的就变成1
    have00=0#反应物有不存在于模型中的就变成1
    have1=0#产物有不能生成的就变成1
    have11=0#产物有不存在于模型中的就变成1
    reactants=[]
    products=[]
    reaction=i[2].replace('<=>', '-->')
    reaction=reaction.replace('<--', '-->')
    reaction=reaction.split('-->')
    if not reaction[0]=='':
        reactants=reaction[0].split(' ')
        for j in reactants:
            if not j=='+' and not j=='' and not re.compile(r'\d+.\d').findall(j):#'\d'代表任意字符，+代表重复多次,匹配系数
                with model:
                    if j in mets.keys():
                        met = str(mets[j])
                        product = model.metabolites.get_by_id(met)
                        demand = model.add_boundary(product, type='demand')  # add demand reaction as the objective
                        model.objective = demand
                        fluxes = cobra.flux_analysis.pfba(model).fluxes
                        a='DM_'+met
                        if float(fluxes[a])<1e-6:
                            have0=1
                            if i[0] not in rea.keys():
                                rea.setdefault(i[0],[]).append(j)
                            else:
                                rea.get(i[0]).append(j)
                    else:
                        have00=1
                        if i[0] not in rea0.keys():
                            rea0.setdefault(i[0],[]).append(j)
                        else:
                            rea0.get(i[0]).append(j)
        if have0==1 and float(i[1])>0:
            print(i[0]+'\t'+i[2]+'\t'+ 'left is 0' + str(rea[i[0]]))
        if have0==1 and float(i[1])<0:
            print(i[0]+'\t'+i[2]+'\t'+ 'right is 0'+ str(rea[i[0]]))
        if have00==1 and float(i[1])>0:
            print(i[0]+'\t'+i[2]+'\t'+ 'left is not in our model' + str(rea0[i[0]]))
        if have00==1 and float(i[1])<0:
            print(i[0]+'\t'+i[2]+'\t'+ 'right is not in our model'+ str(rea0[i[0]]))
    if not reaction[1]=='':
        products=reaction[1].split(' ')
        for j1 in products:
            if not j1=='+' and not j=='' and not re.compile(r'\d+.\d').findall(j1):#'\d'代表任意字符，+代表重复多次,匹配系数
                with model:
                    if j1 in mets.keys():
                        met1 = str(mets[j1])
                        product = model.metabolites.get_by_id(met1)
                        demand = model.add_boundary(product, type='demand')  # add demand reaction as the objective
                        model.objective = demand
                        fluxes = cobra.flux_analysis.pfba(model).fluxes
                        a1='DM_'+met1
                        if float(fluxes[a1])<1e-6:
                            have1=1
                            if i[0] not in rea1.keys():
                                rea1.setdefault(i[0],[]).append(j1)
                            else:
                                rea1.get(i[0]).append(j1)
                    else:
                        have11=1
                        if i[0] not in rea11.keys():
                            rea11.setdefault(i[0],[]).append(j1)
                        else:
                            rea11.get(i[0]).append(j1)
        if have1==1 and float(i[1])>0:
            print(i[0]+'\t'+i[2]+'\t'+ 'right is 0' + str(rea1[i[0]]))
        if have1==1 and float(i[1])<0:
            print(i[0]+'\t'+i[2]+'\t'+ 'left is 0'+ str(rea1[i[0]]))  
        if have11==1 and float(i[1])>0:
            print(i[0]+'\t'+i[2]+'\t'+ 'right is not in our model' + str(rea11[i[0]]))
        if have11==1 and float(i[1])<0:
            print(i[0]+'\t'+i[2]+'\t'+ 'left is not in our model'+ str(rea11[i[0]]))  
       # print(products)
        

R01221	 Glycine_c0 + NAD_c0 + Tetrahydrofolate_c0 --> 5_10_Methylenetetrahydrofolate_c0 + CO2_c0 + H_c0 + NADH_c0 + NH3_c0	left is 0['Tetrahydrofolate_c0']
rxn00691	 10_Formyltetrahydrofolate_c0 + H2O_c0 --> Formate_c0 + H_c0 + Tetrahydrofolate_c0	left is 0['10_Formyltetrahydrofolate_c0']
rxn03031	 CoA_c0 + N_Succinyl_L_2_amino_6_oxopimelate_c0 <-- H2O_c0 + Succinyl_CoA_c0 + tetrahydrodipicolinate_c0	right is 0['N_Succinyl_L_2_amino_6_oxopimelate_c0']
DM_L_Lysine_c0	 L_Lysine_c0 -->	left is 0['L_Lysine_c0']


In [64]:
a='2.0'
b=r'\d'
#s = re.match(b,a)
print(re.compile(r'\d+.\d').findall(a))#'\d'代表任意字符，+代表重复多次
print(re.match(r'\d',a))
print(re.findall(r'\d{2}','21c34d56e78'))

['2.0']
<_sre.SRE_Match object; span=(0, 1), match='2'>
['21', '34', '56', '78']


In [165]:
mets={}
for m in model.metabolites:
    mets[m.name]=m
   
print(mets['2S-4S-4-hydroxy-2-3-4-5-tetrahydrodipicolinate'])

KeyError: '2S-4S-4-hydroxy-2-3-4-5-tetrahydrodipicolinate'

In [129]:
host_dict={}
#host_dict['rea']='1'
host_dict.setdefault('rea',[]).append('1')
host_dict.get('rea').append('s')
#a.get('rea').append(2)
#    if host is not '':
#      if host_dict.has_key(name):
#        host_dict.get(name).append(host)#此处为关键向字典里已经有的key(name)值后继续添加value(host)
#      else:
 #       host_dict.setdefault(name,[]).append(host)#创建{name，[host]}value为列表的格式的字典。
print(host_dict['rea'])
if 'rea' in host_dict.keys():
    print('yes')

['1', 's']
yes


In [103]:
for j in reactants:
            if not j=='+' and not re.compile(r'\d+.\d').findall(j):#'\d'代表任意字符，+代表重复多次,匹配系数
                print(j)
print(mets['Pyruvate_c0'])


L_Aspartate4_semialdehyde_c0
Pyruvate_c0

cpd00020_c0


In [169]:
if 'Pyruvate_c0' in mets.keys():
    print(mets.keys())

dict_keys(['5_Methylthioadenosine_c0', 'D_3_Hydroxydodecanoyl_acp_c0', 'Dianteisoheptadecanoylphosphatidylethanolamine_c0', 'Citrate_Mg_e0', 'K_c0', '6_methyl_3_hydroxy_heptanoyl_ACP_c0', 'gly_glu_L_c0', 'trdox_c0', 'gamma_Glutamylcysteine_c0', 'Salicin_6P_c0', '2E_Dodecenoyl_acp_c0', '2_Octaprenyl_3_methyl_6_methoxy_1_4_benzoquinone_c0', 'L_Methionine_e0', 'dGDP_c0', 'Acetoacetyl_ACP_c0', 'hexadecenoate_c0', 'Dihydrofolate_c0', 'L_Lysine_c0', 'Undecaprenylphosphate_c0', 'GDP_mannose_c0', 'D_Serine_c0', 'Orotidylic_acid_c0', '1_isoheptadecanoyl_sn_glycerol_3_phosphate_c0', 'dCDP_c0', 'Oxaloacetate_c0', '8_methyl_3_oxo_decanoyl_ACP_c0', 'Maltopentaose_c0', 'L_3_Amino_isobutyrate_c0', '1_2_Diisohexadecanoyl_sn_glycerol_c0', 'Lauroyl_KDO2_lipid_IVA_c0', 'Dianteisopentadecanoylphosphatidylglycerophosphate_c0', 'L_Aspartate_c0', '3_oxodecanoyl_acp_c0', 'fa1_c0', 'Urea_e0', 'Tartronate_semialdehyde_c0', '6_methyl_3_oxo_octanoyl_ACP_c0', 'H2S2O3_e0', 'Gly_Phe_c0', 'L_Lactaldehyde_c0', '3_4_di

In [144]:
inputfolder = './PST_mets/' # Folder's storing all calculated results of biomass precursors
check_pre=inputfolder+'L_Lysine_c0.txt' # the precursors we want to check 
s=csv.reader(open(check_pre),delimiter="\t")
realist=[]
realist.extend(s)
mets={} #字典存放代谢物ID和全称对应列表
for m in model.metabolites:
    mets[m.name]=m
for i in realist:
    rea={}
    have0=0
    have1=0
    reactants=[]
    products=[]
    reaction=i[2].replace('<=>', '-->')
    reaction=reaction.split('-->')
    reactants=reaction[0].split(' ')
    #products=reaction[1].split(' ')
    print(reaction[1])

 2S-4S-4-hydroxy-2-3-4-5-tetrahydrodipicolinate + H2O_c0
 H2O_c0 + NAD_c0 + tetrahydrodipicolinate_c0
 ADP_c0 + D_Glucose_c0 + H_c0 + Phosphate_c0
 5_10_Methylenetetrahydrofolate_c0 + CO2_c0 + H_c0 + NADH_c0 + NH3_c0
 2.0 H_c0 + NADPH_c0 + NAD_c0
 H2O_c0 + Phosphoenolpyruvate_c0
 3_Phosphonooxypyruvate_c0 + H_c0 + NADH_c0
 ATP_c0 + H2O_c0 + 3.0 H_c0
 6_phospho_D_glucono_1_5_lactone_c0 + H_c0 + NADPH_c0
 L_Glutamate_c0 + Oxaloacetate_c0
 meso_2_6_Diaminopimelate_c0
 LL_2_6_Diaminopimelate_c0 + Succinate_c0
 2_Keto_3_deoxy_6_phosphogluconate_c0 + H2O_c0
 2.0 H_c0 + Oxaloacetate_c0 + Phosphate_c0
 ADP_c0 + D_glucose_6_phosphate_c0
 10_Formyltetrahydrofolate_c0 + H_c0
 Glyceraldehyde3_phosphate_c0 + Pyruvate_c0
 CO2_c0 + NADH_c0
 NH3_c0
 1_3_Bisphospho_D_glycerate_c0 + NADH_c0
 3_Phosphonooxypyruvate_c0 + L_Glutamate_c0
 L_Serine_c0 + Tetrahydrofolate_c0
 5_10_Methenyltetrahydrofolate_c0 + NADPH_c0
 CO2_c0 + L_Lysine_c0
 ADP_c0 + Phosphate_c0 + Succinyl_CoA_c0
 ADP_c0 + H_c0 + Oxaloacetate

IndexError: list index out of range

In [149]:
a='L_Lysine_c0 -->'.split('-->')
b=a[1].split(' ')
print(b[0])

In [184]:
#5.4 Gap check for seed model, according to the calculated results of curated model finding the gap.
inputfolder = './PST_mets/' # Folder's storing all calculated results of biomass precursors
check_pre=inputfolder+'L_Arginine_c0.txt' # the precursors we want to check 
s=csv.reader(open(check_pre),delimiter="\t")
realist=[]
realist.extend(s)
mets={} #字典存放代谢物ID和全称对应列表
for m in model.metabolites:
    mets[m.name]=m
for i in realist:
    rea={} #存放模型中存在的反应物
    rea1={}#存放模型中存在的产物
    rea0={} #存放模型中不存在的反应物
    rea11={}#存放模型中不存在的产物
    have0=0 #反应物有不能生成的就变成1
    have00=0#反应物有不存在于模型中的就变成1
    have1=0#产物有不能生成的就变成1
    have11=0#产物有不存在于模型中的就变成1
    reactants=[]
    products=[]
    reaction=i[2].replace('<=>', '-->')
    reaction=reaction.replace('<--', '-->')
    reaction=reaction.split('-->')
    if not reaction[0]=='':
        reactants=reaction[0].split(' ')
        for j in reactants:
            if not j=='+' and not re.compile(r'\d+.\d').findall(j):#'\d'代表任意字符，+代表重复多次,匹配系数
                with model:
                    if j in mets.keys():
                        met = str(mets[j])
                        product = model.metabolites.get_by_id(met)
                        demand = model.add_boundary(product, type='demand')  # add demand reaction as the objective
                        model.objective = demand
                        fluxes = cobra.flux_analysis.pfba(model).fluxes
                        a='DM_'+met
                        if float(fluxes[a])<1e-6:
                            have0=1
                            if i[0] not in rea.keys():
                                rea.setdefault(i[0],[]).append(j)
                            else:
                                rea.get(i[0]).append(j)
        if have0==1 and float(i[1])>0:
            print(i[0]+'\t'+i[2]+'\t'+ 'left is 0' + str(rea[i[0]]))
        if have0==1 and float(i[1])<0:
            print(i[0]+'\t'+i[2]+'\t'+ 'right is 0'+ str(rea[i[0]]))
    if not reaction[1]=='':
        products=reaction[1].split(' ')
        for j1 in products:
            if not j1=='+' and not re.compile(r'\d+.\d').findall(j1):#'\d'代表任意字符，+代表重复多次,匹配系数
                with model:
                    if j1 in mets.keys():
                        met1 = str(mets[j1])
                        product = model.metabolites.get_by_id(met1)
                        demand = model.add_boundary(product, type='demand')  # add demand reaction as the objective
                        model.objective = demand
                        fluxes = cobra.flux_analysis.pfba(model).fluxes
                        a1='DM_'+met1
                        if float(fluxes[a1])<1e-6:
                            have1=1
                            if i[0] not in rea1.keys():
                                rea1.setdefault(i[0],[]).append(j1)
                            else:
                                rea1.get(i[0]).append(j1)
        if have1==1 and float(i[1])>0:
            print(i[0]+'\t'+i[2]+'\t'+ 'right is 0' + str(rea1[i[0]]))
        if have1==1 and float(i[1])<0:
            print(i[0]+'\t'+i[2]+'\t'+ 'left is 0'+ str(rea1[i[0]]))  
        

rxn01637	 2_Acetamido_5_oxopentanoate_c0 + L_Glutamate_c0 --> 2_Oxoglutarate_c0 + N_Acetylornithine_c0	right is 0['N_Acetylornithine_c0']
rxn00802	 L_Argininosuccinate_c0 <=> Fumarate_c0 + L_Arginine_c0	left is 0['L_Argininosuccinate_c0']
rxn00802	 L_Argininosuccinate_c0 <=> Fumarate_c0 + L_Arginine_c0	right is 0['L_Arginine_c0']
rxn01434	 ATP_c0 + Citrulline_c0 + L_Aspartate_c0 <=> AMP_c0 + L_Argininosuccinate_c0 + PPi_c0	left is 0['Citrulline_c0']
rxn01434	 ATP_c0 + Citrulline_c0 + L_Aspartate_c0 <=> AMP_c0 + L_Argininosuccinate_c0 + PPi_c0	right is 0['L_Argininosuccinate_c0']
rxn01019	 Carbamoylphosphate_c0 + Ornithine_c0 <=> Citrulline_c0 + 2.0 H_c0 + Phosphate_c0	left is 0['Ornithine_c0']
rxn01019	 Carbamoylphosphate_c0 + Ornithine_c0 <=> Citrulline_c0 + 2.0 H_c0 + Phosphate_c0	right is 0['Citrulline_c0']
rxn01636	 L_Glutamate_c0 + N_Acetylornithine_c0 <=> N_Acetyl_L_glutamate_c0 + Ornithine_c0	left is 0['N_Acetylornithine_c0']
rxn01636	 L_Glutamate_c0 + N_Acetylornithine_c0 <=> N